# Writing SQW for BIFROST

In [ ]:
import scipp as sc
import sciline
from scippneutron.io import sqw
from pathlib import Path
import scippnexus as snx
import numpy as np
import dataclasses

from ess import bifrost
from ess.bifrost.data import (
    simulated_elastic_incoherent_with_phonon,
    tof_lookup_table_simulation
)
from ess.spectroscopy.types import *

In [ ]:
n_det = 3
n_angle = 5

bin_sizes = {'u1': 6, 'u2': 7, 'u3': 8, 'u4': 9}

out_file = Path("bifrost-simulated.sqw")

# Q projections
u = sc.vector([1, 0, 0], unit="1/angstrom")
v = sc.vector([0, 1, 0], unit="1/angstrom")
w = sc.cross(u, v)  # must be orthogonal to u and v for now

In [ ]:
with snx.File(simulated_elastic_incoherent_with_phonon()) as f:
    detector_names = list(f['entry/instrument'][snx.NXdetector])
detector_names = detector_names[:n_det]

workflow = bifrost.BifrostSimulationWorkflow(detector_names)
workflow[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()
workflow[TimeOfFlightLookupTable] = sc.io.load_hdf5(tof_lookup_table_simulation())
workflow[PreopenNeXusFile] = PreopenNeXusFile(True)
scheduler = sciline.scheduler.NaiveScheduler()

data = workflow.compute(EnergyData[SampleRun], scheduler=scheduler)

data = data['a3', :n_angle].flatten(['triplet', 'tube', 'length'], 'detector')

In [ ]:
settings = sc.DataArray(
    sc.arange('s', data.sizes['a3']*data.sizes['a4'], unit=None).fold('s', sizes={'a3': data.sizes['a3'], 'a4': data.sizes['a4']}),
    coords={
        'a3': data.coords['a3'],
        'a4': data.coords['a4'],
    }
).flatten(to='setting')

In [ ]:
def project_onto(direction, vec):
    return sc.dot(direction / sc.norm(direction), vec)

aux = (data
            .flatten(['a3', 'a4'], 'setting')
            # Split into two calls because of https://github.com/scipp/scipp/issues/3766
            .transform_coords(
                u1=lambda sample_table_momentum_transfer: project_onto(u, sample_table_momentum_transfer),
                u2=lambda sample_table_momentum_transfer: project_onto(v, sample_table_momentum_transfer),
                u3=lambda sample_table_momentum_transfer: project_onto(w, sample_table_momentum_transfer),
                keep_inputs=False,
            ).transform_coords(u4="energy_transfer", keep_inputs=False)
            .bins.drop_coords(['incident_energy', 'incident_wavelength', 'lab_momentum_transfer'])
            .drop_coords(['a3', 'a4', 'secondary_flight_time'])
         )
binned = (aux
          .bins.assign_coords({
            'setting': sc.bins_like(aux, settings.data),
            'detector_number': sc.bins_like(aux, aux.coords.pop('detector_number')),
          })
          .bins.concat()
          .bin(bin_sizes)
          )

In [ ]:
def coord_range(da: sc.DataArray, coord: str) -> sc.Variable:
    assert sc.islinspace(da.coords[coord])
    return sc.array(dims=[coord], values=[da.coords[coord][0].value, da.coords[coord][-1].value],
                    unit=da.coords[coord].unit)

In [ ]:
sample = sqw.SqwIXSample(
    name="Vibranium",
    lattice_spacing=sc.vector([2.86, 2.86, 2.86], unit="angstrom"),
    lattice_angle=sc.vector([90.0, 90.0, 90.0], unit="deg"),
)

In [ ]:
dnd_metadata = sqw.SqwDndMetadata(
    axes=sqw.SqwLineAxes(
        title="My Axes",
        label=["u1", "u2", "u3", "u4"],
        img_scales=[
            sc.scalar(1.0, unit="1/angstrom"),
            sc.scalar(1.0, unit="1/angstrom"),
            sc.scalar(1.0, unit="1/angstrom"),
            sc.scalar(1.0, unit="meV"),
        ],
        img_range=[
            coord_range(binned, name)
            for name in ["u1", "u2", "u3", "u4"]
        ],
        n_bins_all_dims=sc.array(dims=["axis"], values=[binned.sizes[f'u{i}'] for i in range(1, 5)], unit=None),
        single_bin_defines_iax=sc.array(dims=["axis"], values=[True] * 4),
        dax=sc.arange("axis", 4, unit=None),
        offset=[
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="meV"),
        ],
        changes_aspect_ratio=True,
    ),
    proj=sqw.SqwLineProj(
        title="My Projection",
        lattice_spacing=sample.lattice_spacing,
        lattice_angle=sample.lattice_angle,
        offset=[
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="1/angstrom"),
            sc.scalar(0.0, unit="meV"),
        ],
        label=["u1", "u2", "u3", "u4"],
        u=u,
        v=v,
        w=None,
        non_orthogonal=False,
        type="aaa",
    ),
)

In [ ]:
dnd_metadata.axes.img_range

In [ ]:
instrument = sqw.SqwIXNullInstrument(
    name="BIFROST",
    source=sqw.SqwIXSource(
        name="ESS",
        target_name="Tungsten wheel",
        frequency=sc.scalar(14, unit="Hz"),
    ),
)

In [ ]:
en = binned.coords['u4'].broadcast(sizes={'detector': data.sizes['detector'], 'u4': len(binned.coords['u4'])}).rename(u4='energy_transfer')
experiment_template = sqw.SqwIXExperiment(
    run_id=0,
    efix=data.coords['final_energy'],
    emode=sqw.EnergyMode.indirect,
    en=en,
    psi=sc.scalar(0.0, unit="rad"),
    u=u,
    v=v,
    omega=sc.scalar(0.0, unit="rad"),
    dpsi=sc.scalar(0.0, unit="rad"),
    gl=sc.scalar(0.0, unit="rad"),
    gs=sc.scalar(0.0, unit="rad"),
)
assert np.unique(data.coords['a4'].values).size == 1
experiments = [
    dataclasses.replace(experiment_template, run_id=i, psi=a3)
    for i, a3 in enumerate(data.coords['a3'])
]

In [ ]:
binned

In [ ]:
# TODO add 0 observations
#  we lose some bins in the above binning processes, e.g., here, idet is only a subset of all detector numbers

In [ ]:
observations = binned.copy().drop_coords(['sample_position', 'source_position'])
observations = observations.bins.assign_coords(
    {
        'idet': observations.bins.coords.pop('detector_number').to(dtype='float32'),
        'irun': observations.bins.coords.pop('setting').to(dtype='float32'),
    }
)
observations = observations.assign_coords(
    {
        f'u{i}': sc.midpoints(observations.coords.pop(f'u{i}')).to(dtype='float32', copy=False)
        for i in range(1, 5)
    }
)
observations.coords['ien'] = sc.arange('u4', observations.sizes['u4'], dtype='float32', unit=None)
observations

In [ ]:
buffer = None

def get_buffer(n: int):
    global buffer
    if buffer is None or buffer.shape[0] < n:
        buffer = np.empty((n, 9), dtype=np.float32)
        return buffer
    return buffer[:n]

buffers = []
for obs_bin in observations.flatten(to='u'):
    h = obs_bin.group('idet', 'irun').hist()
    n = h.sizes['idet'] * h.sizes['irun']
    h = h.flatten(to='obs')
    buf = get_buffer(n)
    buf[:, 0] = h.coords['u1'].values
    buf[:, 1] = h.coords['u2'].values
    buf[:, 2] = h.coords['u3'].values
    buf[:, 3] = h.coords['u4'].values
    buf[:, 4] = h.coords['irun'].values
    buf[:, 5] = h.coords['idet'].values
    buf[:, 6] = h.coords['ien'].values
    buf[:, 7] = h.values
    buf[:, 8] = sc.stddevs(h).values
    buffers.append(buf.copy())

pix_buffer = np.concat(buffers, axis=0)

In [ ]:
bin_sizes = sc.array(dims=['u'], values=[buf.shape[0] for buf in buffers], unit=None)
bin_sizes = bin_sizes.fold(dim='u', sizes=observations.sizes)

In [ ]:
builder = sqw.Sqw.build(out_file, title="Simulated data with phonon").add_default_instrument(instrument).add_default_sample(sample)
builder = builder.add_dnd_data(dnd_metadata, data=observations.hist().data, counts=bin_sizes)
builder = builder.add_pixel_data(pix_buffer, experiments=experiments)
builder.create()

In [ ]:
with sqw.Sqw.open(out_file) as f:
    print(f.data_block_names())
    d = f.read_data_block('data', "nd_data")

In [ ]:
d[0].shape

In [ ]:
binned.hist().data['u1', 3]['u2', 6]['u3', 1]['u4', 0]

In [ ]:
# TODO The dim order is reversed.
# Seems wrong, I thought we handled transposition properly now?!
d[0][0][1][6][3]

In [ ]:
d[0][3][6][1][0]

In [ ]:
np.nonzero(d[0] == 127.0)